In [ ]:
!pip -q install transformers==4.17
!pip -q install optuna
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.17.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 M

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import pandas as pd
import nltk
import re
import ast
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
import torch.nn as nn
import optuna
import time

In [ ]:
splits = {'python_train': 'data/python_train-00000-of-00001.parquet', 'python_test': 'data/python_test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_train"])
test_df = pd.read_parquet("hf://datasets/NLBSE/nlbse25-code-comment-classification/" + splits["python_test"])

display(train_df.head())
display(test_df.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,index,class,comment_sentence,partition,combo,labels
0,1,AccessMixin,functionality.,0,functionality. | AccessMixin,"[0, 0, 0, 0, 1]"
1,8,Atomic,"when it s used as a decorator, call wraps the ...",0,"when it s used as a decorator, call wraps the ...","[1, 0, 0, 0, 0]"
2,10,Atomic,"when it s used as a context manager, enter cre...",0,"when it s used as a context manager, enter cre...","[1, 0, 0, 0, 0]"
3,12,Atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,Atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,AccessMixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,AmbiguityError,more than one migration matches a name prefix.,1,more than one migration matches a name prefix....,"[0, 0, 0, 0, 1]"
2,3,AppConfigStub,stub of an appconfig.,1,stub of an appconfig. | AppConfigStub,"[0, 0, 0, 0, 1]"
3,4,AppConfigStub,only provides a label and a dict of models.,1,only provides a label and a dict of models. | ...,"[0, 1, 0, 0, 0]"
4,5,Archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [ ]:
def remove_punctuation_except_pipe(text):
    return re.sub(r"[^\w\s\|]", "", text)

# Apply the function to the 'combo' column
train_df['combo'] = train_df['combo'].apply(remove_punctuation_except_pipe)
test_df['combo'] = test_df['combo'].apply(remove_punctuation_except_pipe)

# Convert data to lowercase
train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)
test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(
    lambda x: x.lower() if isinstance(x, str) else str(x)
)

train_df[["class", "comment_sentence", "combo"]] = train_str_df[["class", "comment_sentence", "combo"]]
test_df[["class", "comment_sentence", "combo"]] = test_str_df[["class", "comment_sentence", "combo"]]

display(train_df.head())
display(test_df.head())

<ipython-input-4-df4392ae2e99>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_str_df = train_df[["class", "comment_sentence", "combo"]].applymap(
<ipython-input-4-df4392ae2e99>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_str_df = test_df[["class", "comment_sentence", "combo"]].applymap(


,index,class,comment_sentence,partition,combo,labels
0,1,accessmixin,functionality.,0,functionality | accessmixin,"[0, 0, 0, 0, 1]"
1,8,atomic,"when it s used as a decorator, call wraps the ...",0,when it s used as a decorator call wraps the e...,"[1, 0, 0, 0, 0]"
2,10,atomic,"when it s used as a context manager, enter cre...",0,when it s used as a context manager enter crea...,"[1, 0, 0, 0, 0]"
3,12,atomic,exit commits the transaction or releases the s...,0,exit commits the transaction or releases the s...,"[1, 0, 0, 0, 0]"
4,14,atomic,it s possible to disable the creation of savep...,0,it s possible to disable the creation of savep...,"[1, 0, 0, 0, 0]"


,index,class,comment_sentence,partition,combo,labels
0,0,accessmixin,abstract cbv mixin that gives access mixins th...,1,abstract cbv mixin that gives access mixins th...,"[0, 0, 0, 0, 1]"
1,2,ambiguityerror,more than one migration matches a name prefix.,1,more than one migration matches a name prefix ...,"[0, 0, 0, 0, 1]"
2,3,appconfigstub,stub of an appconfig.,1,stub of an appconfig | appconfigstub,"[0, 0, 0, 0, 1]"
3,4,appconfigstub,only provides a label and a dict of models.,1,only provides a label and a dict of models | a...,"[0, 1, 0, 0, 0]"
4,5,archive,the external api class that encapsulates an ar...,1,the external api class that encapsulates an ar...,"[0, 0, 1, 0, 1]"


In [ ]:
X = list(train_df['combo'])
y = list(train_df['labels'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=5)

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/59.8M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, device="cpu"):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        encoded_text = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = encoded_text['input_ids'].squeeze().to(self.device)
        attention_mask = encoded_text['attention_mask'].squeeze().to(self.device)
        label = torch.tensor(label, dtype=torch.float).to(self.device)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

train_dataset = TextClassificationDataset(X_train, y_train, tokenizer)
eval_dataset = TextClassificationDataset(X_val, y_val, tokenizer)

In [ ]:
class OneHotTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
        outputs = model(**model_inputs)
        logits = outputs.get("logits")

        if logits is None:
            raise ValueError("Logits are missing from the model output.")

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits, labels.float())

        # Log training loss
        if self.state.global_step % self.args.logging_steps == 0:
            print(f"Step {self.state.global_step}, Training Loss: {loss.item()}")

        return (loss, outputs) if return_outputs else loss


# class OneHotTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         # Remove 'labels' from inputs before passing to model
#         model_inputs = {k: v for k, v in inputs.items() if k != "labels"}
#         outputs = model(**model_inputs)  # Pass only the required inputs to the model
#         logits = outputs.get("logits")

#         if logits is None:
#             # Handle the case where logits are missing, e.g., raise an exception or return a default loss
#             raise ValueError("Logits are missing from the model output.")  # Or return a default loss

#         # Use BCEWithLogitsLoss for multi-label classification
#         loss_fct = nn.BCEWithLogitsLoss()
#         loss = loss_fct(logits, labels.float())

#         return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(pred):
    logits = pred.predictions
    preds = (logits > 0).astype(int)
    labels = pred.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=1)
    accuracy = accuracy_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# def compute_metrics(pred):
#     logits = pred.predictions
#     preds = (logits > 0).astype(int)

#     labels = pred.label_ids

#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     accuracy = (preds == labels).mean()

#     return {
#         'accuracy': accuracy,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1
#     }


In [ ]:
def objective(trial):
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 20)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)

    # training_args = TrainingArguments(
    #     output_dir="./results",
    #     num_train_epochs=num_train_epochs,
    #     per_device_train_batch_size=batch_size,
    #     per_device_eval_batch_size=batch_size * 2,
    #     warmup_steps=10,
    #     weight_decay=weight_decay,
    #     evaluation_strategy="epoch",
    #     learning_rate=learning_rate,
    #     gradient_accumulation_steps=2,
    #     report_to="none",
    # )

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size * 2,
        warmup_steps=10,
        weight_decay=weight_decay,
        evaluation_strategy="epoch",
        logging_dir="./logs",  # Directory for logging
        logging_steps=10,  # Log every 10 steps
        save_steps=500,
        learning_rate=learning_rate,
        gradient_accumulation_steps=2,
        report_to="none",  # Avoid reporting to third-party loggers like WandB
        logging_first_step=True,  # Log the first step
    )

    trainer = OneHotTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    print("Starting training...")
    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best trial:")
print(study.best_trial)

[I 2024-12-01 15:53:23,644] A new study created in memory with name: no-name-8c45ec8b-ae4f-47d6-808a-54d8ed54529c
<ipython-input-10-747f1883d862>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-5)
<ipython-input-10-747f1883d862>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-4, 0.1)


Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 940


Step 0, Training Loss: 0.6907334923744202
Step 0, Training Loss: 0.6918603181838989


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.582200,0.571854,0.000000,1.000000,0.000000,0.000000
1,0.533400,0.521292,0.000000,1.000000,0.000000,0.000000
2,0.523300,0.509048,0.000000,1.000000,0.000000,0.000000
3,0.507800,0.505790,0.000000,1.000000,0.000000,0.000000
4,0.509200,0.501989,0.000000,1.000000,0.000000,0.000000
5,0.499500,0.496842,0.000000,1.000000,0.000000,0.000000
6,0.486400,0.489918,0.000000,1.000000,0.000000,0.000000
7,0.475900,0.485373,0.000000,1.000000,0.000000,0.000000
8,0.486800,0.483023,0.000000,1.000000,0.000000,0.000000
9,0.481300,0.481083,0.000000,1.000000,0.000000,0.000000


...

***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


Step 330, Training Loss: 0.0008603844908066094
Step 330, Training Loss: 0.0005630910163745284
Step 340, Training Loss: 0.0006920182495377958
Step 340, Training Loss: 0.00040861821616999805
Step 350, Training Loss: 0.0006494270637631416
Step 350, Training Loss: 0.0012426788453012705
Step 360, Training Loss: 0.0005317928153090179
Step 360, Training Loss: 0.00042957710684277117
Step 370, Training Loss: 0.004443936515599489
Step 370, Training Loss: 0.0004019782063551247


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 377
  Batch size = 32


[I 2024-12-01 16:23:26,701] Trial 19 finished with value: 0.703597522054273 and parameters: {'learning_rate': 2.176329965042299e-06, 'batch_size': 16, 'num_train_epochs': 8, 'weight_decay': 0.015273142582206321}. Best is trial 14 with value: 0.7065333724623482.


Best trial:
FrozenTrial(number=14, state=TrialState.COMPLETE, values=[0.7065333724623482], datetime_start=datetime.datetime(2024, 12, 1, 16, 15, 37, 997274), datetime_complete=datetime.datetime(2024, 12, 1, 16, 16, 49, 853177), params={'learning_rate': 4.961806220526654e-05, 'batch_size': 4, 'num_train_epochs': 7, 'weight_decay': 0.09875385997446333}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-06, step=None), 'batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'num_train_epochs': IntDistribution(high=20, log=False, low=3, step=1), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.0001, step=None)}, trial_id=14, value=None)


In [ ]:
best_hyperparams = study.best_trial.params

training_args = TrainingArguments(
    output_dir="./best_results",
    num_train_epochs=best_hyperparams["num_train_epochs"],
    per_device_train_batch_size=best_hyperparams["batch_size"],
    per_device_eval_batch_size=best_hyperparams["batch_size"] * 2,
    learning_rate=best_hyperparams["learning_rate"],
    weight_decay=best_hyperparams["weight_decay"],
    warmup_steps=10,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2,
    report_to="none",
)

trainer = OneHotTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Measure training time
print("Starting training...")
start_time = time.time()  # Start the timer

trainer.train()

end_time = time.time()  # End the timer
training_duration = end_time - start_time  # Calculate the duration

# Print the training duration
print(f"Training completed in {training_duration:.2f} seconds.")

PyTorch: setting up devices
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1507
  Num Epochs = 7
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1316


Starting training...
Step 0, Training Loss: 0.0005677736480720341
Step 0, Training Loss: 0.00019565853290259838


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.919494,0.620690,0.694112,0.694581,0.686467
1,No log,0.906385,0.623342,0.707778,0.682266,0.682748
2,0.010500,0.987548,0.618037,0.697830,0.662562,0.668768
3,0.010500,0.963170,0.641910,0.713637,0.689655,0.694927
4,0.010500,0.995234,0.625995,0.705441,0.677340,0.686664
5,0.006100,0.975010,0.636605,0.702263,0.687192,0.690891
6,0.006100,0.959731,0.628647,0.707609,0.682266,0.689437


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-500
Configuration saved in ./best_results/checkpoint-500/config.json
Model weights saved in ./best_results/checkpoint-500/pytorch_model.bin


Step 500, Training Loss: 0.0004373354895506054
Step 500, Training Loss: 0.00030765365227125585


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
Saving model checkpoint to ./best_results/checkpoint-1000
Configuration saved in ./best_results/checkpoint-1000/config.json
Model weights saved in ./best_results/checkpoint-1000/pytorch_model.bin


Step 1000, Training Loss: 0.0003291229368187487
Step 1000, Training Loss: 0.00022600183729082346


***** Running Evaluation *****
  Num examples = 377
  Batch size = 8
***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




Training completed in 69.15 seconds.


In [ ]:
eval_results = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 377
  Batch size = 8


### Model Saving

In [ ]:
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_python'
!mkdir 'best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

# Save model and tokenizer
trainer.save_model('./best_model_huawei-noah_TinyBERT_General_4L_312D_python')
tokenizer.save_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer')

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python')
tokenizer = AutoTokenizer.from_pretrained('./best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer')

mkdir: cannot create directory ‘best_model_huawei-noah/TinyBERT_General_4L_312D_python’: File exists
mkdir: cannot create directory ‘best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer’: File exists


Saving model checkpoint to ./best_model_huawei-noah/TinyBERT_General_4L_312D_python
Configuration saved in ./best_model_huawei-noah/TinyBERT_General_4L_312D_python/config.json
Model weights saved in ./best_model_huawei-noah/TinyBERT_General_4L_312D_python/pytorch_model.bin
tokenizer config file saved in ./best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/tokenizer_config.json
Special tokens file saved in ./best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/special_tokens_map.json
loading configuration file ./best_model_huawei-noah/TinyBERT_General_4L_312D_python/config.json
Model config BertConfig {
  "_name_or_path": "./best_model_huawei-noah/TinyBERT_General_4L_312D_python",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": 

### Model Testing

In [ ]:
X_test = list(test_df['combo'])
y_test = list(test_df['labels'])

In [ ]:
df_test = pd.DataFrame({"combo":X_test,"labels":y_test})
test_text = df_test.combo.values
test_label = df_test.labels.values

print(test_label)

[array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 1, 0, 0, 0]) array([0, 0, 1, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([1, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 1, 0]) array([1, 0, 0, 0, 0])
 array([1, 0, 0, 0, 0]) array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1])
 array([1, 0, 0, 0, 0]) array([1, 0, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0])
 array([0, 1, 0, 0, 0]) array([0, 1, 0, 0, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0]) array([0, 0, 0, 1, 0])
 array([0, 0, 1, 0, 0]) array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1])
 array([0, 0, 0, 0, 1]) array([0, 0, 0, 0, 1]) array([0, 0, 1, 0, 0])
 array([0, 0, 0, 0, 

In [ ]:
test_dataset = TextClassificationDataset(test_text, test_label, tokenizer)

# Measure inference time
start_time = time.time()

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Calculate elapsed time
end_time = time.time()
inference_time = end_time - start_time

# Compute metrics using the `compute_metrics` function
metrics = compute_metrics(predictions)

# Display metrics and inference time
print("Evaluation Metrics:", metrics)
print(f"Inference Time: {inference_time:.2f} seconds")

***** Running Prediction *****
  Num examples = 406
  Batch size = 8


Evaluation Metrics: {'accuracy': 0.5566502463054187, 'precision': 0.6591005454171888, 'recall': 0.6100917431192661, 'f1': 0.6288028934457701}
Inference Time: 0.95 seconds


In [ ]:
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_python'
!zip -r best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip './best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python/ (stored 0%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python/pytorch_model.bin (deflated 7%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python/training_args.bin (deflated 51%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python/config.json (deflated 52%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/ (stored 0%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/tokenizer_config.json (deflated 42%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/special_tokens_map.json (deflated 40%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/tokenizer.json (deflated 71%)
updating: best_model_huawei-noah/TinyBERT_General_4L_312D_python_tokenizer/vocab.txt (deflated 53%)


In [ ]:
# Transferring the model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

!cp best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/"
!cp best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip "/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot create regular file '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah/TinyBERT_General_4L_312D_Python/': No such file or directory
cp: cannot create regular file '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah/TinyBERT_General_4L_312D_Python_Tokenizer/': No such file or directory


### Load and Test Model

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'path/to/checkpoint-folder' with the actual path to your checkpoint folder.
model_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/'
tokenizer_folder = '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/'

!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip' -d './'
!unzip '/content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip' -d './'

unzip:  cannot find or open /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip, /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip.zip or /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python/best_model_huawei-noah_TinyBERT_General_4L_312D_python.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip, /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer.zip.zip or /content/drive/MyDrive/FYP/Revised/Models/huawei-noah_TinyBERT_General_4L_312D_Python_Tokenizer/best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokeniz

In [6]:
best_model_huawei_noah_TinyBERT_General_4L_312D_python = 'best_model_huawei-noah_TinyBERT_General_4L_312D_python'
best_model_huawei_noah_TinyBERT_General_4L_312D_python_tokenizer = 'best_model_huawei-noah_TinyBERT_General_4L_312D_python_tokenizer'

In [7]:
# Load the model and tokenizer from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_python)
tokenizer = AutoTokenizer.from_pretrained(best_model_huawei_noah_TinyBERT_General_4L_312D_python_tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: best_model_huawei-noah_TinyBERT_General_4L_312D_python is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
text = "for detecting automatically generated fields."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
output

SequenceClassifierOutput(loss=None, logits=tensor([[-7.7154, -8.3863,  8.6639, -7.5060, -8.8254]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
logits = output.logits

# Apply sigmoid to convert logits to probabilities
probabilities = torch.sigmoid(logits)

# Define a threshold to determine if a class is positive
threshold = 0.5
predicted_indices = (probabilities > threshold).nonzero(as_tuple=True)[1]

# Convert indices to a list for output
predicted_classes = (predicted_indices + 1).tolist()

# Format the output to display classes
if len(predicted_classes) > 0:
    predicted_classes_str = ", ".join(map(str, predicted_classes))
    print(f"Predicted class/es: {predicted_classes_str}")
else:
    print("No positive classes predicted.")

Predicted class: 3
